### VGG 모델
- 2014년 영국 옥스포드대학에서 개발됨
- Video Gemetru Group
- 단순하지만 깊게 학습함
- 이전 다른 CNN 모델보다 복잡하고 다양한 크기의 필터를 사용
- 무조건 작은 3x3 필터만 반복해서 쌓아줌
- layer를 많이 쌓아 올려서 깊게 만들어주는 원리를 적용
- VGG16, VGG19 -> 숫자=layer의 개수

[주요 로직]  
입력 이미지(예: 고양이, 강아지 사진)  
↓  
[3x3 필터] => 색 덩어리 파악  
↓  
[3x3 필터 여러번 사용] => 선, 모양 파악  
↓  
[3x 필터 더 깊게] => 귀, 눈, 수염같은 특징 파악  
↓  
[Fully Connected] => "고양이"라는 결론

[단계별 구현]
- 1단계: 데이터 준비/로딩
- 2단계: VGG16 불러오기
- 3단계: 새 분류기를 추가
    > Flatten -> Dense(256, relu) -> Dropout(0.5) -> Dense(10, softmax)  
    > Cifar-10 분류에 맞게끔 Fully Connected Layer 재구성
- 4단계: 학습 전략
    > Optimizer: Adam
    > Loss: categorical_crossentropy
    > metices: accuracy
- 5단계: 데이터 증강(Data Augmentation)
    > 회전, 이동, 좌/우 반전 등을 적용시켜 모델의 일반화 성능 향상
- 6단계: 학습
    > datagen.flow()로 배치 증강 학습
    > epochs: 20(필요에 따라서 조정)
- 7단계: 평가
    > Test 데이터셋으로 최종 성능 평가
- 8단계: accuracy, Loss를 시각화


In [ ]:
# 라이브러리 불러오기
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
# 1. 데이터 준비 (예: CIFAR-10)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0
# 데이터 shape 확인
x_train.shape, y_train.shape
# 라벨 원-핫 인코딩
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# VGG16 기반 Feature Extractor 불러오기
base_model = VGG16(
    weights='imagenet', # 사전학습 된 가중치
    include_top=False,  # FC 레이어 제외, =True: 전체 VGG16을 가져옴(ImageNet용 분류기 포함)
    input_shape=(32, 32, 3))
# 모든 가중치를 동결(fine-tuning 전에 feature extractor로 활용)
for layer in base_model.layers:
    layers.trainable = False
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax") # 숫자 0~9 분류
])
# 모델 컴파일
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=15,      # 사진을 좌/우로 최대 +-15도 회전
    width_shift_range=0.1,  # 좌/우로 최대 10%
    height_shift_range=0.1, # 상/하로 최대 10%
    horizontal_flip=True    # 좌/우 반전 허용
)
datagen.fit(x_train)
# 모델 학습
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs=10,
    validation_data=(x_test, y_test)
    )
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_tset, verbose=2)
print(f"테스트 정확도: {test_acc:.4f}")
# 학습 결과 시각화
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('VGG16 Transfer Learning Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()